In [ ]:
import rospy
from sensor_msgs.msg import PointCloud2, Image, PointField
import sensor_msgs.point_cloud2 as pc2
from std_msgs.msg import Header
from velodyne_msgs.msg import VelodyneScan
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from cv_bridge import CvBridge
import time
import cv2
import open3d as o3d

In [ ]:
ZED_TOPIC = "/islam/zed_pts"
VLP_TOPIC = "/velodyne_points"
ZED_DEPTH = '/zed/zed_node/depth/depth_registered'

In [ ]:
ZED_V = 360
ZED_H = 640
ZED_ANGLE = 90
LiDAR_V = 16

In [ ]:
rospy.init_node('sf', anonymous=True)
# zed_s = rospy.wait_for_message(ZED_TOPIC, PointCloud2)
# vlp_s = rospy.Subscriber(VLP_TOPIC, PointCloud2, vlp_callback)

In [ ]:
def sph_to_cart_pts(pts):
    pts[:,1] = cp.radians(pts[:,1])
    pts[:,2] = cp.radians(pts[:,2])

    # Convert spherical coordinates to Cartesian coordinates
    x = pts[:, 0] * cp.cos(pts[:, 1]) * cp.cos(pts[:, 2])
    y = pts[:, 0] * cp.cos(pts[:, 1]) * cp.sin(pts[:, 2])
    z = pts[:, 0] * cp.sin(pts[:, 1])
    
    return cp.asarray([x, y, z]).T

def cart_to_sph_pts(pts):
    # Convert to CuPy array
    pts = cp.asarray(pts)

    # Convert to spherical coordinates
    x, y, z = pts[:, 0], pts[:, 1], pts[:, 2]
    r = cp.sqrt(x**2 + y**2 + z**2)
    theta = cp.arctan(z / cp.sqrt(x**2 + y**2))
    phi = cp.arctan(y / x)

    return cp.column_stack((r, cp.degrees(theta), cp.degrees(phi)))
    # return appendSpherical_np(pts)[:,3:6]

def msg2pts(msg):
    return np.array(list(pc2.read_points(msg, field_names=("x", "y", "z"))))

In [ ]:
vlp_filtered_p = rospy.Publisher("/islam/vlp_filt", PointCloud2, queue_size=50)

def callback(msg):
    start = time.time()
    vlp_pts = msg2pts(msg)
    vlp_sph_pts_raw = cart_to_sph_pts(vlp_pts[vlp_pts[:,0] > 0])
    mask = (vlp_sph_pts_raw[:, 2] < ZED_ANGLE/2) & (vlp_sph_pts_raw[:, 2] > -ZED_ANGLE/2)
    vlp_sph_pts = vlp_sph_pts_raw[mask]
    
    header = Header()
    header.stamp = rospy.Time.now()
    header.frame_id = 'map'
    
    msg_p = pc2.create_cloud_xyz32(header, sph_to_cart_pts(vlp_sph_pts).get()) 

    vlp_filtered_p.publish(msg_p)    

rospy.Subscriber(VLP_TOPIC, PointCloud2, callback)
rospy.spin()

In [ ]:
# zed_pts = rospy.wait_for_message(ZED_TOPIC, PointCloud2)
zed_img = rospy.wait_for_message(ZED_DEPTH, Image)
vlp_pts = rospy.wait_for_message(VLP_TOPIC, PointCloud2)
if zed_img and vlp_pts:
    print('OK')

In [ ]:
bridge = CvBridge()
zed_depth = np.array(bridge.imgmsg_to_cv2(zed_img, "32FC1"))

In [ ]:
vlp_pts = msg2pts(vlp_pts)
vlp_sph_pts_raw = cart_to_sph_pts(vlp_pts[vlp_pts[:,0] > 0])
mask = (vlp_sph_pts_raw[:, 2] < ZED_ANGLE/2) & (vlp_sph_pts_raw[:, 2] > -ZED_ANGLE/2)
vlp_sph_pts = vlp_sph_pts_raw[mask]

In [ ]:
min(vlp_sph_pts[:,1])

In [ ]:
print(np.nanmean(zed_depth[10]))

In [ ]:
vlp_depth = zed_depth

row_theta_range = 2
row_theta_max = 15

for i in range(0, LiDAR_V):
    mask = (vlp_sph_pts[:, 1] < row_theta_max - i*row_theta_range) & (vlp_sph_pts[:, 2] > (row_theta_max - (i+1)*row_theta_range))
    row = vlp_sph_pts[mask]
    # print(row[:,2])
    # print(len(vlp_sph_pts[mask]))
    for col in row:
        # print(vlp_depth[int(i*180/LiDAR_V),int(col[2]*ZED_ANGLE/ZED_H)])
        vlp_depth[int(i*180/LiDAR_V),int(col[2]*ZED_ANGLE/ZED_H)] = col[0]
        # print(vlp_depth[int(i*180/LiDAR_V),int(col[2]*ZED_ANGLE/ZED_H)])
    
    print(row[:,0].mean())


In [ ]:
plt.imshow(vlp_depth)
vlp_depth.shape

In [ ]:
plt.imshow(zed_depth)
zed_depth.shape

In [ ]:
def lpf(img, ncutoff):
    # Apply 2D FFT to the image
    f = cp.fft.fft2(img)

    # Shift the zero frequency component to the center of the spectrum
    fshift = cp.fft.fftshift(f)

    # Create a circular mask of the same size as the spectrum
    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    cutoff = int(min(crow, ccol)*ncutoff)
    cv2.circle(mask, (ccol, crow), cutoff, 1, -1)
    # cv2.ellipse(mask, (ccol, crow), (1, 2) * cutoff, 0, 0, 360,  1, -1)

    mask = cp.asarray(mask)

    # Apply the mask to the shifted spectrum
    fshift_filtered = fshift * mask

    # Shift the zero frequency component back to the corner of the spectrum
    f_filtered = cp.fft.ifftshift(fshift_filtered)

    # Apply the inverse 2D FFT to the filtered spectrum
    img_filtered = cp.fft.ifft2(f_filtered)
    img_filtered = cp.real(img_filtered)

    return img_filtered


def pg(input, us_rate, ncutoff, threshold=100):
    ncutoff = ncutoff / 10
    filtered = input

    while threshold > 0:
        filtered = lpf(filtered, ncutoff)
        filtered[::us_rate, ::us_rate] = input[::us_rate, ::us_rate]

        threshold -= 1

    return filtered

In [ ]:
m = ZED_V//LiDAR_V + 1

pg_frame_init = zed_depth
pg_frame_init[::m, :] = vlp_depth

In [ ]:
plt.imshow(pg_frame_init)
plt.show()

In [ ]:
start = time.time()
pg_frame = pg(cp.asarray(pg_frame_init), m, ncutoff=1, threshold=50)
print(time.time() - start)

In [ ]:
plt.imshow(pg_frame.get())
plt.show()